In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import altair as alt
import polars as pl


In [69]:
from plan2eplus.studies.analysis2.all_cases_v_time import get_temp_dfs, get_ventilation_and_wind_speed_dfs
from plan2eplus.studies.experiments.retrieve import retrieve_control_cases

In [68]:
cases = retrieve_control_cases()
df, df_site = get_temp_dfs(cases)

df_agg = (
    df.group_by(["case_names", "datetimes"])
    .agg(pl.mean("Zone Mean Air Temperature [C]"))
    .join(
        df_site.select(["datetimes", "Site Outdoor Air Drybulb Temperature [C]"]),
        on=["datetimes"],
    )
    .with_columns(site_legend_entry=pl.lit("Site"))
)
df_agg

case_names,datetimes,Zone Mean Air Temperature [C],Site Outdoor Air Drybulb Temperature [C],site_legend_entry
str,datetime[μs],f64,f64,str
"""bol_5_Medium""",2017-07-01 12:15:00,24.977844,26.4,"""Site"""
"""bol_5_Medium""",2017-07-01 12:45:00,25.844737,27.4,"""Site"""
"""red_b1_Medium""",2017-07-01 19:30:00,24.575157,23.075,"""Site"""
"""red_b1_Medium""",2017-07-01 13:15:00,27.270234,28.1,"""Site"""
"""bol_5_Medium""",2017-07-01 16:30:00,27.320443,27.05,"""Site"""
…,…,…,…,…
"""bol_5_Medium""",2017-07-01 06:30:00,19.330845,16.55,"""Site"""
"""red_b1_Medium""",2017-07-01 14:00:00,27.971943,28.6,"""Site"""
"""red_b1_Medium""",2017-07-01 02:00:00,17.726257,15.0,"""Site"""


In [73]:
df, df_site = get_ventilation_and_wind_speed_dfs(cases)

df_agg_vent = (
    df.group_by(["case_names", "datetimes"])
    .agg([pl.mean("AFN Zone Ventilation Volume [m3]"), pl.mean("AFN Zone Mixing Volume [m3]")])
    .join(
        df_site.select(["datetimes", "Site Wind Speed [m/s]"]),
        on=["datetimes"],
    )
    .with_columns(site_legend_entry=pl.lit("Site"))
)
df_agg_vent

case_names,datetimes,AFN Zone Ventilation Volume [m3],AFN Zone Mixing Volume [m3],Site Wind Speed [m/s],site_legend_entry
str,datetime[μs],f64,f64,f64,str
"""bol_5_Medium""",2017-07-01 12:15:00,503.0499,354.925289,3.95,"""Site"""
"""red_b1_Medium""",2017-07-01 23:00:00,224.006423,285.467586,1.725,"""Site"""
"""red_b1_Medium""",2017-07-01 05:45:00,76.309386,122.18204,0.0,"""Site"""
"""bol_5_Medium""",2017-07-01 15:15:00,737.812246,565.982489,5.25,"""Site"""
"""bol_5_Medium""",2017-07-01 03:45:00,96.933978,123.483107,0.5,"""Site"""
…,…,…,…,…,…
"""bol_5_Medium""",2017-07-01 10:15:00,285.556549,265.40647,3.4,"""Site"""
"""amb_b1_Medium""",2017-07-01 10:15:00,92.883281,432.478185,3.4,"""Site"""
"""red_b1_Medium""",2017-07-01 22:00:00,252.186047,319.406887,1.95,"""Site"""


In [185]:
def convert_title(title:str):
    title = title.replace('\\', "").replace(":Q", "")
    if "m/s" in title:
        return title.replace("m/s", "ᵐ⁄ₛ")
    if "m3" in title:
        return title.replace("m/s", "m³")
    return title

In [183]:
domain = ['Site', "amb_b1_Medium", "bol_5_Medium", "red_b1_Medium"]
range_ = ['black', 'seagreen', 'firebrick', 'rebeccapurple']

def plot_col(df, site_var, case_var, second_axis=False):
    """site_var and case_var must be raw strings according to altair encoding docs! """
    base = alt.Chart(df).mark_line().encode(x=alt.X("datetimes:T").title("Time of Day"))
    
    site = base.mark_line(color="black").encode(
        y = alt.Y(site_var, title=convert_title(site_var)),
        color=alt.Color("site_legend_entry").scale(domain=domain, range=range_)
    )

    if second_axis:
        site.encode(
            alt.Y(site_var, title=convert_title(site_var)).scale(domain=(0,7), clamp=True),
        )
    
    cases = base.encode(
        y=alt.Y(case_var, title=convert_title(case_var)).scale(zero=False),
        color=alt.Color("case_names:N"),
    )
    
    tick_axis = alt.Axis(labels=False, domain=False, ticks=False)
    
    box = (
        alt.Chart(df)
        .mark_boxplot(extent="min-max")
        .encode(
            y=alt.Y("case_names:N", axis=tick_axis, title=""),
            x=alt.X(case_var, title=convert_title(case_var)).scale(zero=False),
            color="case_names",
        )
    )

    if second_axis:
        return (cases + site).resolve_scale(y="independent") & box
    
    else:
        return (cases + site).resolve_scale(y="shared") & box

In [186]:
dfs = [df_agg_vent]*2  + [df_agg] 
site_vars =[r"Site Wind Speed \[m/s\]:Q"]*2 +  [r"Site Outdoor Air Drybulb Temperature \[C\]:Q"] 
case_vars = [r"AFN Zone Ventilation Volume \[m3\]",r"AFN Zone Mixing Volume \[m3\]" , r"Zone Mean Air Temperature \[C\]:Q"]
second_axis = [True]*2 + [False]

chart = alt.hconcat()
for df, site_var, case_var, ax2 in zip(dfs, site_vars, case_vars, second_axis):
    chart |= plot_col(df, site_var, case_var, ax2)

chart

alt.HConcatChart(...)

In [132]:
base = alt.Chart(df_agg_vent).mark_line().encode(x="datetimes:T")


cases = base.encode(
    alt.Y(r"AFN Zone Ventilation Volume \[m3\]").scale(zero=False),
    color=alt.Color("case_names:N"),
)

site = base.mark_line(color="black").encode(
    # alt.Y(r"Site Outdoor Air Drybulb Temperature \[C\]:Q"),
    alt.Y(r"Site Wind Speed \[m/s\]:Q").scale(domain=(0,7), clamp=True),
    color=alt.Color("site_legend_entry").scale(domain=domain, range=range_)

)
(cases + site).resolve_scale(y="independent")

alt.LayerChart(...)

In [106]:


base = alt.Chart(df_agg).mark_line().encode(x="datetimes:T")


cases = base.encode(
    alt.Y(r"Zone Mean Air Temperature \[C\]:Q").scale(zero=False),
    color=alt.Color("case_names:N"),
)

site = base.mark_line(color="black").encode(
    # alt.Y(r"Site Outdoor Air Drybulb Temperature \[C\]:Q"),
    alt.Y(r"Site Outdoor Air Drybulb Temperature \[C\]:Q"),
    color=alt.Color("site_legend_entry").scale(domain=domain, range=range_)

)

tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

box = (
    alt.Chart(df_agg)
    .mark_boxplot(extent="min-max")
    .encode(
        y=alt.Y("case_names:N", axis=tick_axis, title=""),
        x=alt.X(r"Zone Mean Air Temperature \[C\]").scale(zero=False),
        color="case_names",
    )
)


(cases + site).resolve_scale("independent")  # + site

alt.LayerChart(...)

In [39]:
alt.Chart(df_agg).mark_boxplot(extent='min-max').encode(
    y=alt.Y('case_names:N'),
    x=alt.X(r'Zone Mean Air Temperature \[C\]').scale(zero=False),
    color="case_names",
)

alt.Chart(...)

In [118]:
from vega_datasets import data

source = data.seattle_weather()

base = alt.Chart(source).encode(
    alt.X('month(date):T').title(None)
)

area = base.mark_area(opacity=0.3, color='#57A44C').encode(
    alt.Y('average(temp_max)').axis(title='Avg. Temperature (°C)', titleColor='#57A44C'),
    alt.Y2('average(temp_min)')
)

line = base.mark_line(stroke='#5276A7', interpolate='monotone').encode(
    alt.Y('average(precipitation)').axis(title='Precipitation (inches)', titleColor='#5276A7')
)

alt.layer(area, line).resolve_scale(
    y='independent'
)

alt.LayerChart(...)